In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import tkinter
%pylab inline
import pandas as pd
from itertools import cycle, islice
from functools import reduce
from scipy.stats import linregress
import scipy.stats as st
import numpy as np

# Study data files
games_details_path = "data_set/games_details.csv"
games_path = "data_set/games.csv"
teams_path = "data_set/teams.csv"


# Read the games data and the study results
games_details = pd.read_csv(games_details_path)
games = pd.read_csv(games_path)
teams = pd.read_csv(teams_path)

Populating the interactive namespace from numpy and matplotlib


In [48]:
games_details.head(3)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,42000102,1610612764,WAS,Washington,203078,Bradley Beal,F,NaN,34:36,14.0,...,0.0,4.0,4.0,3.0,1.0,0.0,1.0,0.0,33.0,-22.0
1,42000102,1610612764,WAS,Washington,1629060,Rui Hachimura,F,NaN,25:50,4.0,...,2.0,5.0,7.0,1.0,0.0,0.0,3.0,4.0,11.0,-6.0
2,42000102,1610612764,WAS,Washington,203458,Alex Len,C,NaN,12:11,3.0,...,3.0,2.0,5.0,0.0,1.0,0.0,0.0,2.0,6.0,-11.0


In [49]:
games.head(3)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-05-26,42000102,Final,1610612755,1610612764,2020,1610612755,120.0,0.557,0.684,...,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1
1,2021-05-26,42000132,Final,1610612752,1610612737,2020,1610612752,101.0,0.383,0.739,...,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1
2,2021-05-26,42000142,Final,1610612762,1610612763,2020,1610612762,141.0,0.544,0.774,...,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1


In [50]:
#Merge the DataFrames
game_gamedetails_merged = pd.merge(games[['GAME_ID','GAME_DATE_EST','SEASON','HOME_TEAM_ID','HOME_TEAM_WINS']], games_details, right_on=['GAME_ID'],left_on=['GAME_ID'], how='inner')
game_gamedetails_merged.head(10)

,GAME_ID,GAME_DATE_EST,SEASON,HOME_TEAM_ID,HOME_TEAM_WINS,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,203078,Bradley Beal,...,0.0,4.0,4.0,3.0,1.0,0.0,1.0,0.0,33.0,-22.0
1,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,1629060,Rui Hachimura,...,2.0,5.0,7.0,1.0,0.0,0.0,3.0,4.0,11.0,-6.0
2,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,203458,Alex Len,...,3.0,2.0,5.0,0.0,1.0,0.0,0.0,2.0,6.0,-11.0
3,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,203526,Raul Neto,...,1.0,2.0,3.0,1.0,0.0,0.0,0.0,2.0,4.0,-17.0
4,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,201566,Russell Westbrook,...,2.0,4.0,6.0,11.0,0.0,0.0,4.0,0.0,10.0,-16.0
5,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,202722,Davis Bertans,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,-11.0
6,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,202397,Ish Smith,...,0.0,1.0,1.0,3.0,0.0,0.0,1.0,1.0,10.0,-10.0
7,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,1629655,Daniel Gafford,...,4.0,2.0,6.0,1.0,1.0,1.0,1.0,1.0,11.0,-1.0
8,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,1628990,Chandler Hutchison,...,0.0,2.0,2.0,1.0,0.0,0.0,1.0,1.0,4.0,-17.0
9,42000102,2021-05-26,2020,1610612755,1,1610612764,WAS,Washington,201577,Robin Lopez,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,-12.0


In [51]:
game_gamedetails_2018 = game_gamedetails_merged.loc[game_gamedetails_merged['SEASON'] == 2018]
game_gamedetails = game_gamedetails_2018.loc[game_gamedetails_2018['GAME_ID'] >= 21800001]

In [52]:
team_point_agg = game_gamedetails.\
groupby(['TEAM_ID'])[['PTS']].\
agg('sum').reset_index()

team_point_agg_df = team_point_agg.sort_values(by='PTS', ascending=False)

team_point_agg_df.head(5)

,TEAM_ID,PTS
7,1610612744,12161.0
24,1610612761,11942.0
12,1610612749,11378.0
20,1610612757,11157.0
18,1610612755,10749.0


In [71]:
TEAM_ID_home = []
games_2018 = games.loc[games['SEASON']== 2018]
games_2018 = games_2018.loc[games_2018['GAME_ID'] >= 21800001]
games_2018.rename(columns={'HOME_TEAM_ID':'TEAM_ID'}, inplace =True)
home_wins = games_2018[['TEAM_ID','GAME_ID','HOME_TEAM_WINS']]
home_wins.head(3)

,TEAM_ID,GAME_ID,HOME_TEAM_WINS
17678,1610612744,41800406,0
17679,1610612761,41800405,0
17680,1610612744,41800404,0


In [73]:
home_wins_agg = home_wins.\
groupby(['TEAM_ID'])[['HOME_TEAM_WINS']].\
agg('sum').reset_index()

home_wins_agg_df = home_wins_agg.sort_values(by='HOME_TEAM_WINS', ascending=False)

home_wins_agg_df.head(5)

,TEAM_ID,HOME_TEAM_WINS
24,1610612761,41
6,1610612743,39
12,1610612749,39
20,1610612757,37
7,1610612744,36


In [65]:
## Which stadium has the most points/ total games.

In [23]:
teams.head(3)

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate


In [93]:
#Merge the DataFrames
team_Points = pd.merge(team_point_agg_df, teams [['TEAM_ID','CITY','ABBREVIATION','ARENA','NICKNAME']], right_on=['TEAM_ID'],left_on=['TEAM_ID'], how='inner')
team_Points.head(10)

,TEAM_ID,PTS,CITY,ABBREVIATION,ARENA,NICKNAME
0,1610612744,12161.0,Golden State,GSW,Chase Center,Warriors
1,1610612761,11942.0,Toronto,TOR,Scotiabank Arena,Raptors
2,1610612749,11378.0,Milwaukee,MIL,Fiserv Forum,Bucks
3,1610612757,11157.0,Portland,POR,Moda Center,Trail Blazers
4,1610612755,10749.0,Philadelphia,PHI,Wells Fargo Center,76ers
5,1610612743,10603.0,Denver,DEN,Pepsi Center,Nuggets
6,1610612745,10535.0,Houston,HOU,Toyota Center,Rockets
7,1610612738,10135.0,Boston,BOS,TD Garden,Celtics
8,1610612746,10130.0,Los Angeles,LAC,Staples Center,Clippers
9,1610612760,9913.0,Oklahoma City,OKC,Chesapeake Energy Arena,Thunder


In [95]:
#Merge the DataFrames
teams_city = pd.merge( team_Points ,home_wins_agg_df , on=['TEAM_ID'], how='left')
teams_city_df = teams_city.sort_values(by='HOME_TEAM_WINS', ascending=False)
teams_city_df

,TEAM_ID,PTS,CITY,ABBREVIATION,ARENA,NICKNAME,HOME_TEAM_WINS
1,1610612761,11942.0,Toronto,TOR,Scotiabank Arena,Raptors,41
2,1610612749,11378.0,Milwaukee,MIL,Fiserv Forum,Bucks,39
5,1610612743,10603.0,Denver,DEN,Pepsi Center,Nuggets,39
3,1610612757,11157.0,Portland,POR,Moda Center,Trail Blazers,37
0,1610612744,12161.0,Golden State,GSW,Chase Center,Warriors,36
6,1610612745,10535.0,Houston,HOU,Toyota Center,Rockets,36
4,1610612755,10749.0,Philadelphia,PHI,Wells Fargo Center,76ers,35
10,1610612759,9879.0,San Antonio,SAS,AT&T Center,Spurs,34
12,1610612762,9650.0,Utah,UTA,Vivint Smart Home Arena,Jazz,30
7,1610612738,10135.0,Boston,BOS,TD Garden,Celtics,30


In [96]:
teams_city_df.to_csv("teams_city.csv", index=False)